# Основное задание
Возьмите датасет https://www.kaggle.com/ajayrana/hymenoptera-data/kernels  
1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)  
2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)  
3.Добавьте аугментацию данных к пункту 2  
  
Сравните качество всех 3 полученных подходов  
  
# Задание со звездочкой  
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?  

import libraries

In [1]:
import torch
from torch import nn
import torchvision.models as models

import torchvision as tv
import time

import os

In [2]:
dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [3]:
dev

device(type='cuda')

helpers

load data

In [4]:
data_dir = '../dll_hw_4_modern_cnn/hymenoptera_data/'

In [5]:
BATCH_SIZE = 32

In [6]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': tv.transforms.Compose([
        tv.transforms.RandomResizedCrop(224),
        tv.transforms.RandomHorizontalFlip(),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': tv.transforms.Compose([
        tv.transforms.Resize(256),
        tv.transforms.CenterCrop(224),
        tv.transforms.ToTensor(),
        tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: tv.datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


create model

In [7]:
# import torchvision.models as models
# resnet18 = models.resnet18()
# alexnet = models.alexnet()
# vgg16 = models.vgg16()
# squeezenet = models.squeezenet1_0()
# densenet = models.densenet161()
# inception = models.inception_v3()
# googlenet = models.googlenet()
# shufflenet = models.shufflenet_v2_x1_0()
# mobilenet = models.mobilenet_v2()
# resnext50_32x4d = models.resnext50_32x4d()
# wide_resnet50_2 = models.wide_resnet50_2()
# mnasnet = models.mnasnet1_0()

In [8]:
# import torchvision.models as models
# resnet18 = models.resnet18(pretrained=True)
# alexnet = models.alexnet(pretrained=True)
# squeezenet = models.squeezenet1_0(pretrained=True)
# vgg16 = models.vgg16(pretrained=True)
# densenet = models.densenet161(pretrained=True)
# inception = models.inception_v3(pretrained=True)
# googlenet = models.googlenet(pretrained=True)
# shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
# mobilenet = models.mobilenet_v2(pretrained=True)
# resnext50_32x4d = models.resnext50_32x4d(pretrained=True)
# wide_resnet50_2 = models.wide_resnet50_2(pretrained=True)
# mnasnet = models.mnasnet1_0(pretrained=True)

# 1.Обучите на нем модели ResNet 18 и VGG 16 с нуля (5-10 эпох)

# ResNet 18

In [9]:
model = models.resnet18()

train model

In [10]:
model = model.to(dev)

In [11]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [12]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [13]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 3.731. Train acc: 0.000. Train Loss: 223.353
Step. time since epoch: 3.794. Train acc: 0.375. Train Loss: 226.487
Step. time since epoch: 3.857. Train acc: 0.594. Train Loss: 141.302
Step. time since epoch: 3.921. Train acc: 0.500. Train Loss: 132.527
Step. time since epoch: 3.989. Train acc: 0.469. Train Loss: 128.991
Step. time since epoch: 4.053. Train acc: 0.562. Train Loss: 65.717
Step. time since epoch: 4.119. Train acc: 0.562. Train Loss: 44.617
Step. time since epoch: 4.165. Train acc: 0.800. Train Loss: 18.436
epoch 1, loss 4.0223, train acc 0.467, test acc 0.458, time 7.0 sec
Step. time since epoch: 1.905. Train acc: 0.469. Train Loss: 974.624
Step. time since epoch: 2.082. Train acc: 0.438. Train Loss: 106.167
Step. time since epoch: 2.148. Train acc: 0.469. Train Loss: 79.164
Step. time since epoch: 2.214. Train acc: 0.531. Train Loss: 39.481
Step. time since epoch: 2.286. Train acc: 0.438. Train Loss: 203.793
Step. time since epoch: 2.342. Train acc

In [14]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16

In [15]:
model = models.vgg16()

train model

In [16]:
model = model.to(dev)

In [17]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [18]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [19]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.194. Train acc: 0.000. Train Loss: 222.072
Step. time since epoch: 2.409. Train acc: 0.406. Train Loss: 2303.064
Step. time since epoch: 2.619. Train acc: 0.625. Train Loss: 165.540
Step. time since epoch: 2.831. Train acc: 0.344. Train Loss: 2576.320
Step. time since epoch: 3.040. Train acc: 0.500. Train Loss: 195.303
Step. time since epoch: 3.249. Train acc: 0.500. Train Loss: 207.591
Step. time since epoch: 3.460. Train acc: 0.469. Train Loss: 194.175
Step. time since epoch: 3.621. Train acc: 0.500. Train Loss: 78.920
epoch 1, loss 24.3565, train acc 0.414, test acc 0.542, time 6.6 sec
Step. time since epoch: 2.351. Train acc: 0.531. Train Loss: 356.930
Step. time since epoch: 2.569. Train acc: 0.375. Train Loss: 125.205
Step. time since epoch: 2.781. Train acc: 0.531. Train Loss: 152.926
Step. time since epoch: 2.992. Train acc: 0.469. Train Loss: 157.170
Step. time since epoch: 3.204. Train acc: 0.531. Train Loss: 133.441
Step. time since epoch: 3.412. Tr

# 2.Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох)

# ResNet 18 pretrained

In [20]:
model = models.resnet18(pretrained=True)

train model

In [21]:
model = model.to(dev)

In [22]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [23]:
model.fc

Linear(in_features=512, out_features=1000, bias=True)

In [24]:
model.fc = nn.Linear(in_features=512, out_features=2).to(dev)

In [25]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 fc.weight
	 fc.bias


In [26]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [27]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [28]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.023. Train acc: 0.438. Train Loss: 25.014
Step. time since epoch: 2.219. Train acc: 0.500. Train Loss: 24.005
Step. time since epoch: 2.275. Train acc: 0.594. Train Loss: 23.093
Step. time since epoch: 2.394. Train acc: 0.625. Train Loss: 19.624
Step. time since epoch: 2.432. Train acc: 0.562. Train Loss: 21.490
Step. time since epoch: 2.485. Train acc: 0.688. Train Loss: 17.720
Step. time since epoch: 2.515. Train acc: 0.750. Train Loss: 17.392
Step. time since epoch: 2.559. Train acc: 0.600. Train Loss: 12.999
epoch 1, loss 0.6612, train acc 0.594, test acc 0.784, time 5.5 sec
Step. time since epoch: 2.047. Train acc: 0.719. Train Loss: 15.903
Step. time since epoch: 2.136. Train acc: 0.781. Train Loss: 16.223
Step. time since epoch: 2.282. Train acc: 0.812. Train Loss: 14.634
Step. time since epoch: 2.782. Train acc: 0.906. Train Loss: 12.224
Step. time since epoch: 2.812. Train acc: 0.844. Train Loss: 12.527
Step. time since epoch: 2.840. Train acc: 0.812.

In [29]:
# release CUDA
model.eval()
torch.cuda.empty_cache()

# VGG 16 pretrained

In [46]:
model = models.vgg16(pretrained=True)

train model

In [47]:
model = model.to(dev)

In [48]:
## Убираем требование градиента:
for param in model.parameters():
    param.requires_grad = False

In [51]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [52]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=2).to(dev)

In [53]:
print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [54]:
def evaluate_accuracy(data_iter, net, dev):
    acc_sum, n = torch.Tensor([0]).to(dev), 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(dev), y.to(dev)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

In [55]:
def train(net, train_iter, test_iter, trainer, num_epochs, dev):
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            trainer.zero_grad()
            X, y = X.to(dev), y.to(dev)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]
            print("Step. time since epoch: {:.3f}. Train acc: {:.3f}. Train Loss: {:.3f}".format(time.time() -  start,
                (y_hat.argmax(axis=1) == y).sum().item() / y.shape[0], l.item()))
        test_acc = evaluate_accuracy(test_iter, net, dev)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, '
              'time %.1f sec'
              % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc,
                 time.time() - start))

In [56]:
lr, num_epochs = 0.001, 10
trainer = torch.optim.Adam(model.parameters(), lr=lr)
train(model, dataloaders_dict['train'], dataloaders_dict['val'], trainer, num_epochs, dev)

Step. time since epoch: 2.046. Train acc: 0.406. Train Loss: 30.381
Step. time since epoch: 2.129. Train acc: 0.750. Train Loss: 17.031
Step. time since epoch: 2.217. Train acc: 0.781. Train Loss: 16.184
Step. time since epoch: 2.368. Train acc: 0.844. Train Loss: 11.024
Step. time since epoch: 2.439. Train acc: 0.875. Train Loss: 10.705
Step. time since epoch: 2.511. Train acc: 0.938. Train Loss: 7.620
Step. time since epoch: 2.584. Train acc: 0.938. Train Loss: 7.114
Step. time since epoch: 2.639. Train acc: 0.850. Train Loss: 5.080
epoch 1, loss 0.4309, train acc 0.795, test acc 0.948, time 5.4 sec
Step. time since epoch: 1.903. Train acc: 0.969. Train Loss: 3.970
Step. time since epoch: 2.017. Train acc: 0.938. Train Loss: 4.808
Step. time since epoch: 2.095. Train acc: 0.969. Train Loss: 5.213
Step. time since epoch: 2.176. Train acc: 0.938. Train Loss: 8.003
Step. time since epoch: 2.248. Train acc: 0.938. Train Loss: 2.854
Step. time since epoch: 2.316. Train acc: 0.906. Train L

# 3.Добавьте аугментацию данных к пункту 2  
  
(Обучите на нем модели ResNet 18 и VGG 16 с использованием FineTuning (5-10 эпох))
  


# Сравните качество всех 3 полученных подходов  


  
# Задание со звездочкой  
Примените FineTuning ResNet 18 к FashionMnist. Удалось ли увидеть резкое увеличение качества?  